First, load the data,in this notebook we are using the Qitta data from the paper "Machine learning–based feature selection to search stable microbial biomarkers: application to inflammatory bowel disease". The data has been preprocessed in R (refer to the r script in iCloud/UTK/GRA-UTK/SelectMicro/MLonMicrobiome).

There are 3 datasets, the orginal paper combined the 3 datasets together, but not sure how they are combined since they have different columns. The imbalance is an issue if analyzing each dataset independently.

In [1]:
import sys
sys.path.append('../../../Code')
import loadData 
import RunML
import RunML_continue
import FS
import metric

import pandas as pd
import numpy as np
import random
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import pickle

In [2]:
qitta_combine = pd.read_csv('../data/count_table/Qitta_3datasetscombined.csv',index_col=0)

In [3]:
print(qitta_combine.shape)

(1569, 3836)


Extract OTU table and response variable

In [4]:
data = qitta_combine.drop(qitta_combine.columns[0], axis=1)
cols_name = data.columns.tolist()
data = data.values

In [5]:
print(data[:15,:10])
print(np.shape(data))

[[  0   0   0   0   0   0   3   0  34   0]
 [  0   0   0   1   0   0   0   0   9   0]
 [  0   0   1   0   0   0   0   0  27   1]
 [  0   0   1   0   0   0   2   0  36   0]
 [  1   0   0   0   0   0   0   0  13   0]
 [  0   0   0   3   0   0  59   0   1   0]
 [  0   0   0   0   0   0 113   0   0   0]
 [  0   0   0   4   0   0  80   0   0   0]
 [  0   0   0   0   0   0  95   0   2   0]
 [  0   0   0   0   0   0  16   0   2   0]
 [  0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   1   0  17   0]
 [  0   0   0   0   0   0  20   0   8   0]
 [ 12   0   0   0   0  18   0   0   0   0]
 [  0   0   0   0   0   1  12   0   0   0]]
(1569, 3835)


In [6]:
y = qitta_combine.iloc[:, 0].values 

In [7]:
set(y)

{'CD', 'UC', 'nonIBD'}

In [8]:
#change y into 2 levels
y = np.where(np.isin(y, ['CD', 'UC']), 'IBD', y)

In [9]:
y

array(['IBD', 'IBD', 'IBD', ..., 'nonIBD', 'nonIBD', 'nonIBD'],
      dtype=object)

preprocess the OTU table

In [10]:
# 1. abundance matrix--> relative abundance matrix
data=FS.relative_abundance(data)
print(np.shape(data))

(1569, 3835)


In [11]:
print(data[:10,:10])

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.46943574e-05 0.00000000e+00
  1.66536050e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 5.30512422e-06
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  4.77461180e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 3.84791501e-06 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.03893705e-04 3.84791501e-06]
 [0.00000000e+00 0.00000000e+00 3.64794420e-06 0.00000000e+00
  0.00000000e+00 0.00000000e+00 7.29588840e-06 0.00000000e+00
  1.31325991e-04 0.00000000e+00]
 [4.92994547e-06 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  6.40892912e-05 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 9.03908500e-06
  0.00000000e+00 0.00000000e+00 1.77768672e-04 0.00000000e+00
  3.01302833e-06 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e

Perform sigle lable feature selection, this is done by calculating Kruskal-Wallis H-test on each OTU for each environmental factor, each OTU/ASV is given a weighted h score.

In [12]:
# 2. calculating H score for each OTU
weights=FS.OTU_H_Score_fun(data,y)

In [13]:
print(min(weights));print(max(weights));print(len(weights))

0.0
203.57286831967502
3835


In [14]:
weights

array([0.80968858, 0.        , 0.        , ..., 0.        , 2.43216536,
       0.        ])

In [15]:
# check the function
data_test = data.copy()
data_test[data_test<0.01] =0

In [16]:
# Group the data by the unique values in the response variable
groups1 = [data_test[y == group, 0] for group in np.unique(y)]
# Perform the Kruskal-Wallis test
stat, p_value = kruskal(*groups1)
# Store the results in a dictionary
print(stat, p_value)

NameError: name 'kruskal' is not defined

In [ ]:
selectedOTU_index[295:305]

In [ ]:
weights[selectedOTU_index[295:305]]

In [ ]:
# Group the data by the unique values in the response variable
groups1 = [data_test[y == group, 2667] for group in np.unique(y)]
# Perform the Kruskal-Wallis test
stat, p_value = kruskal(*groups1)
# Store the results in a dictionary
print(stat, p_value)

In [ ]:
# 3. determine the elbowpoint and the select indices of the features
selectedOTU_index, eps=FS.indice_H_unisig(weights,y)

In [ ]:
print(eps)

In [ ]:
#plot the h statistics and cutoff descendingly
FS.plotWeightedIndex(weights,threshold=eps)

This plot shows that the gain in information decreases as we include more OTUs along the ranks. 

In [ ]:
# 4. subset the relative matrix by the select indices

In [ ]:
X_FS = data[:,selectedOTU_index]

In [ ]:
np.shape(X_FS)

Prepare 4 datasets: full dataset, our selected dataset, Lasso selected  dataset(based on the target variable), randomly selected data (selected the same numer of variables as in our method, repeat the  process iter=20 times to  find the mean accuracy and AUC)---> use random forest and SVM as classifier

In [ ]:
iter =30
cls = ["RF","SVM"]

In [ ]:
targetLabel=y
X_lasso,xlabel_lasso = RunML_continue.LassoFeatureSelection(data,targetLabel)# this is depend on the target variable(do not update here)
print(np.shape(X_lasso))

In [ ]:
print(xlabel_lasso)

In [ ]:
print(np.shape(data))
print(len(xlabel_lasso))

In [ ]:
data_subset = {"AllFeatures":data, 
               "SelectMicro": X_FS,
               "Lasso":X_lasso,
               "Random":data
              }
print(np.shape(data))
print(np.shape(X_FS))
print(np.shape(X_lasso))

started modeling, the  function will print out the accuracy and AUC for each dataset using each classifier. and also will return the y_actual, y_predict, y_predprob

In [ ]:
# the result of 5-fold cross validation
dict_cm = RunML_continue.runClassifier_FScompare(data_subsets= data_subset,y= targetLabel,N=iter,classifiers=cls)

In [ ]:
# Save dictionary to a pickle file
with open('../results/dict_cm_qitta_update.pkl', 'wb') as pickle_file:
    pickle.dump(dict_cm, pickle_file)

In [ ]:
# Load dictionary from the pickle file
with open('../results/dict_cm_qitta_update.pkl', 'rb') as pickle_file:
    dict_cm = pickle.load(pickle_file)

In [ ]:
# do not use plotmacro_confusion_matrices (not for binary classification)
for dataset_name, classifiers in dict_cm.items():
        for classifier_name, labels in classifiers.items():
            actual_labels = labels[0]
            predicted_labels = labels[1]
            metric.plotmacro_confusion_matrices(actual_labels, predicted_labels,f"{dataset_name} - {classifier_name}")

In [ ]:
selectedASVs0=[cols_name[i] for i in selectedOTU_index]

In [ ]:
selectedASVs0

## compare the first 45 index

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
# the   df with the largest H statistics features
entries=60
selectedOTU_index_15=selectedOTU_index[:entries]
X_FS_15=data[:,selectedOTU_index_15]
df=pd.DataFrame(data=X_FS_15)

In [ ]:
# the column names of the featues
ASVs = cols_name
selectedASVs15=[ASVs[i] for i in selectedOTU_index_15]

In [ ]:
print(set(targetLabel))
RunML.plotPresenseRatio(X_FS_15,targetLabel,selectedASVs15,posLabel="IBD",posText="IBD",negText="nonIBD",entries=entries)

In [ ]:
selectedASV_lasso = [cols_name[i] for i in xlabel_lasso]
RunML.plotPresenseRatio(X_lasso,targetLabel,selectedASV_lasso,posLabel="IBD",posText="IBD",negText="nonIBD",entries=45)

In [ ]:
selectedASV_lasso

In [ ]:
qitta_combine[['Diagnosis','X4414821']].value_counts()

### Analyze the significant OTU

In [ ]:
selectedASVs5=[cols_name[i] for i in selectedOTU_index[:5]]

In [ ]:
qitta_TAX = pd.read_csv('../Data/Qitta_tax.csv',index_col=0)

In [ ]:
selectedASVs5